In [1]:
# addprocs(30);
addprocs(7);

In [4]:
@everywhere begin
    using POMDPModels
    using POMCP
    using POMDPs
    import POMDPToolbox.PreviousObservation
    import POMCP.init_V
    # import POMCP.POMCPPolicy
    import POMDPs: action, updater
end

In [5]:
# N = 100000;
N = 100;
problem = BabyPOMDP(-5, -10);

In [6]:
@everywhere type RandomBabyPolicy <: POMDPs.Policy
    rng::AbstractRNG
end
@everywhere function action(p::RandomBabyPolicy, b::POMDPs.Belief, a=BabyAction(false))
    a.feed = rand(p.rng)>0.5
    return a
end
@everywhere updater(::RandomBabyPolicy) = POMDPToolbox.EmptyUpdater()

In [7]:
function est_reward(problem, policy, belief, N; eps=0.01)
    sum = @parallel (+) for i in 1:N
        sim_rng = MersenneTwister(i)
        sim = POMDPToolbox.RolloutSimulator(rng=sim_rng, initial_state=BabyState(false), eps=eps)
        up = updater(policy)
        POMDPs.simulate(sim, problem, policy, up, convert_belief(up,belief))
    end
    return sum/N;
end

est_reward (generic function with 1 method)

In [8]:
# Feed when Crying (Expected Reward for this nearly-optimal policy is -17.14)
# Test from earlier this week with 5000 experiments: -16.72
@time est_reward(problem, FeedWhenCrying(), PreviousObservation(BabyObservation(false)), N)

-15.061634270536405

1.162281 seconds (482.03 k allocations: 20.724 MB, 0.62% gc time)


This is better than in the crying babies test because epsilon is large and, more importantly, it gets a notcrying observation on the first step every time

In [9]:
# Random
pol_rng = MersenneTwister(7)
@time est_reward(problem, RandomBabyPolicy(pol_rng), POMDPToolbox.EmptyBelief(), N)

-32.237364849049165

0.607114 seconds (426.77 k allocations: 17.946 MB, 0.73% gc time)


In [10]:
# POMCP with FWC rollout policy
# test from earlier this week with 5000 experiments: -16.77
rng = MersenneTwister(2)

solver = POMCPSolver(rollout_policy=FeedWhenCrying(),
                    eps=0.01,
                    c=10.0,
                    tree_queries=300, 
                    rng=rng,
                    updater=updater(problem))
                    

policy = solve(solver, problem)

@time est_reward(problem, policy, BabyStateDistribution(0.0), N)

 28

-14.694925426652963

In [11]:
# POMCP with Random rollout policy
rng = MersenneTwister(2)
rollout_pol_rng = MersenneTwister(2)

solver = POMCPSolver(rollout_policy=RandomBabyPolicy(rollout_pol_rng),
                     eps=0.01,
                     c=10.0,
                     tree_queries=300, 
                     rng=rng,
                     updater=updater(problem))

policy = solve(solver, problem)

@time est_reward(problem, policy, BabyStateDistribution(0.0), N)

.787750 seconds (182.06 k allocations: 8.020 MB, 0.01% gc time)
 20

-15.982194944371333

In [12]:
# Optimal policy for these particular problem parameters:
# if the belief that the baby is hungry is over .28206, then feed (see DMU book)
@everywhere begin
    type OptBabyPolicy <: POMDPs.Policy
    end
    function action(p::OptBabyPolicy, b::BabyStateDistribution, a=BabyAction(false))
        a.feed = b.p_hungry>0.28206
        return a
    end
    updater(::OptBabyPolicy) = updater(BabyPOMDP(-5,-10))
end
@time est_reward(problem, OptBabyPolicy(), BabyStateDistribution(0.0), N)

.300595 seconds (47.35 k allocations: 2.546 MB)
  

-14.686851194997828

0.216601 seconds (47.39 k allocations: 2.562 MB)
